# taxi3_policy_iteration

## taxi3_policy_iteration For First Homework Submission 

In [11]:
import gymnasium as gym
import numpy as np
import time
from contextlib import redirect_stdout
import io

# === Decoding a state into components ===
# Observation space is encoded as an integer, decoded here:
# ((taxi_row * 5 + taxi_col) * 5 + passenger_location) * 4 + destination
# taxi_row: 5 options, taxi_col: 5 options
# passenger_location: 5 (Red, Green, Yellow, Blue, In taxi)
# destination: 4 (Red, Green, Yellow, Blue)
def decode_taxi_state(state):
    taxi_row = (state // 100) % 5
    taxi_col = (state // 20) % 5
    pass_loc = (state // 4) % 5
    dest_idx = state % 4
    return taxi_row, taxi_col, pass_loc, dest_idx

# === Custom map drawing ===
# Shows where the taxi is, and includes layout with labeled destinations.
def draw_custom_map(taxi_row, taxi_col, pass_loc, dest_idx, carrying):
    map_lines = [
        list("+---------+"),
        list("|R: | : :G|"),  # R and G mark Red and Green destination
        list("| : | : : |"),
        list("| : : : : |"),
        list("| | : | : |"),
        list("|Y| : |B: |"),  # Y and B mark Yellow and Blue destination
        list("+---------+")
    ]
    
    for i in range(len(map_lines)):
        while len(map_lines[i]) < 12:
            map_lines[i].append(' ')

    row_map = [1, 2, 3, 4, 5]
    col_map = [1, 3, 6, 9, 11]

    if 0 <= taxi_row < 5 and 0 <= taxi_col < 5:
        vis_row = row_map[taxi_row]
        vis_col = col_map[taxi_col]
        if vis_col < len(map_lines[vis_row]):
            taxi_symbol = "T" if not carrying else "T"
            map_lines[vis_row][vis_col] = taxi_symbol

    return '\n'.join(''.join(line) for line in map_lines)

# === Policy Iteration (solves the MDP) ===
# Covers:
# - State transitions
# - Reward system: -1 step, -10 illegal, +20 dropoff
# - Action effects from env.unwrapped.P
def policy_iteration(env, discount_factor=0.9, max_iterations=1000, theta=1e-5):
    n_states = env.observation_space.n   # Total of 500 possible states
    n_actions = env.action_space.n
    policy = np.zeros(n_states, dtype=int)
    V = np.zeros(n_states)

    for i in range(max_iterations):
        # === Policy Evaluation ===
        while True:
            delta = 0
            for s in range(n_states):
                a = policy[s]
                v = 0
                for prob, next_state, reward, done in env.unwrapped.P[s][a]:
                    v += prob * (reward + discount_factor * V[next_state])
                delta = max(delta, abs(v - V[s]))
                V[s] = v
            if delta < theta:
                break

        # === Policy Improvement ===
        policy_stable = True
        for s in range(n_states):
            old_action = policy[s]
            action_values = np.zeros(n_actions)
            for a in range(n_actions):
                for prob, next_state, reward, done in env.unwrapped.P[s][a]:
                    action_values[a] += prob * (reward + discount_factor * V[next_state])
            best_action = np.argmax(action_values)
            policy[s] = best_action
            if old_action != best_action:
                policy_stable = False

        if policy_stable:
            print(f"\nPolicy converged at iteration {i}")
            break

    return policy, V

# === Mapping actions to text ===
action_names = {
    0: "South ↓",
    1: "North ↑",
    2: "East →",
    3: "West ←",
    4: "Pickup :)",
    5: "Dropoff ⬇"
}

# === Create environment ===
# Covers: Initial state is sampled uniformly from valid set of 300
env = gym.make("Taxi-v3", render_mode=None)
policy, values = policy_iteration(env)

observation, info = env.reset()
done = False
truncated = False
total_reward = 0
step = 0

print("\nStarting Taxi-v3 Simulation...\n")

# === Episode simulation loop with goal clarity ===
locations = ["Red", "Green", "Yellow", "Blue", "In Taxi"]

while not done and not truncated:
    action = policy[observation]
    value = values[observation]

    # Decode observation
    taxi_row, taxi_col, pass_idx, dest_idx = decode_taxi_state(observation)
    carrying = pass_idx == 4  # Passenger is in the taxi if pass_loc == 4

    # Render custom map
    map_output = draw_custom_map(taxi_row, taxi_col, pass_idx, dest_idx, carrying)

    # Display step info
    print(map_output)
    print(f"Step: {step}")
    print(f"State ID: {observation} → (row={taxi_row}, col={taxi_col}, pass_loc={pass_idx}, dest={dest_idx})")
    print(f"Passenger location: {locations[pass_idx]}")
    print(f"Destination: {locations[dest_idx]}")
    print(f"Action: {action_names[action]} ({action})")
    print(f"Value Estimate: {value:.2f}")

    # Take action
    observation, reward, done, truncated, info = env.step(action)
    total_reward += reward
    print(f"Reward received: {reward}")
    print(f"{'-'*30}")
    time.sleep(0.7)
    step += 1
    
print(f"\nEpisode finished in {step} steps with total reward: {total_reward}")
env.close()



Policy converged at iteration 16

Starting Taxi-v3 Simulation...

+---------+ 
|R: | : :G|T
| : | : : | 
| : : : : | 
| | : | : | 
|Y| : |B: | 
+---------+ 
Step: 0
State ID: 93 → (row=0, col=4, pass_loc=3, dest=1)
Passenger location: Blue
Destination: Green
Action: South ↓ (0)
Value Estimate: 24.68
Reward received: -1
------------------------------
+---------+ 
|R: | : :G| 
| : | : : |T
| : : : : | 
| | : | : | 
|Y| : |B: | 
+---------+ 
Step: 1
State ID: 193 → (row=1, col=4, pass_loc=3, dest=1)
Passenger location: Blue
Destination: Green
Action: South ↓ (0)
Value Estimate: 28.54
Reward received: -1
------------------------------
+---------+ 
|R: | : :G| 
| : | : : | 
| : : : : |T
| | : | : | 
|Y| : |B: | 
+---------+ 
Step: 2
State ID: 293 → (row=2, col=4, pass_loc=3, dest=1)
Passenger location: Blue
Destination: Green
Action: South ↓ (0)
Value Estimate: 32.82
Reward received: -1
------------------------------
+---------+ 
|R: | : :G| 
| : | : : | 
| : : : : | 
| | : | : |T
|Y| : |B

## taxi3_policy_iteration For The Final With Metrics

In [3]:
import gymnasium as gym
import numpy as np
import time
from tqdm import tqdm          
import matplotlib.pyplot as plt


# === Decoding a state into components ===
# Observation space is encoded as an integer, decoded here:
# ((taxi_row * 5 + taxi_col) * 5 + passenger_location) * 4 + destination
# taxi_row: 5 options, taxi_col: 5 options
# passenger_location: 5 (Red, Green, Yellow, Blue, In taxi)
# destination: 4 (Red, Green, Yellow, Blue)
def decode_taxi_state(state):
    taxi_row = (state // 100) % 5
    taxi_col = (state // 20) % 5
    pass_loc = (state // 4) % 5
    dest_idx = state % 4
    return taxi_row, taxi_col, pass_loc, dest_idx

# === Custom map drawing ===
# Shows where the taxi is, and includes layout with labeled destinations.
def draw_custom_map(taxi_row, taxi_col, pass_loc, dest_idx, carrying):
    map_lines = [
        list("+---------+"), list("|R: | : :G|"), list("| : | : : |"),
        list("| : : : : |"), list("| | : | : |"), list("|Y| : |B: |"),
        list("+---------+")
    ]
    for i in range(len(map_lines)):
        while len(map_lines[i]) < 12: map_lines[i].append(' ')
    row_map = [1, 2, 3, 4, 5]
    col_map = [1, 3, 6, 9, 11]
    if 0 <= taxi_row < 5 and 0 <= taxi_col < 5:
        vis_row = row_map[taxi_row]
        vis_col = col_map[taxi_col]
        if vis_col < len(map_lines[vis_row]):
            map_lines[vis_row][vis_col] = "T" # Simplified symbol
    return '\n'.join(''.join(line) for line in map_lines)

# === Policy Iteration (solves the MDP) ===
def policy_iteration(env, discount_factor=0.9, max_iterations=1000, theta=1e-5):
    n_states = env.observation_space.n
    n_actions = env.action_space.n
    policy = np.zeros(n_states, dtype=int) # Initialize with a random/arbitrary policy
    V = np.zeros(n_states)
    iterations_converged = -1 # To store convergence iteration

    for i in range(max_iterations):
        # === Policy Evaluation ===
        eval_iter = 0
        while True:
            eval_iter += 1
            delta = 0
            for s in range(n_states):
                v_old = V[s] # Store old V[s] for delta calculation
                a = policy[s]
                v_new = 0
                # env.P[s][a] is a list of (prob, next_state, reward, done) tuples
                for prob, next_state, reward, done in env.P[s][a]:
                    v_new += prob * (reward + discount_factor * V[next_state])
                V[s] = v_new
                delta = max(delta, abs(v_new - v_old))
            if delta < theta:
                # print(f"Policy Evaluation converged in {eval_iter} iterations for PI iteration {i+1}")
                break
            if eval_iter > 10000: 
                print(f"Policy Evaluation did not converge within 10000 iterations for PI iteration {i+1}")
                break


        # === Policy Improvement ===
        policy_stable = True
        for s in range(n_states):
            old_action = policy[s]
            action_values = np.zeros(n_actions)
            for a__check in range(n_actions): # Renamed 'a' to avoid conflict
                for prob, next_state, reward, done in env.P[s][a__check]:
                    action_values[a__check] += prob * (reward + discount_factor * V[next_state])
            best_action = np.argmax(action_values)
            policy[s] = best_action
            if old_action != best_action:
                policy_stable = False
        
        print(f"Policy Iteration: Iteration {i+1} complete. Policy stable: {policy_stable}")

        if policy_stable:
            iterations_converged = i + 1
            print(f"\nPolicy converged at iteration {iterations_converged}")
            break
        if i == max_iterations -1:
             print(f"\nPolicy Iteration reached max_iterations ({max_iterations}) without full stability.")
             iterations_converged = max_iterations


    return policy, V, iterations_converged

# === Evaluate Final Policy Performance ===
def evaluate_policy(env, policy, num_episodes=1000):
    total_rewards = []
    total_steps = []
    successful_dropoffs = 0

    print(f"\n--- Evaluating Final Policy for {num_episodes} episodes ---")
    for i in tqdm(range(num_episodes), desc="Evaluating Policy"):
        observation, _ = env.reset()
        done = False
        truncated = False
        episode_reward = 0
        episode_steps = 0
        
        # Max steps per evaluation episode to prevent infinite loops from a bad policy
        # (though PI should yield an optimal or near-optimal policy)
        for _ in range(200): # Taxi-v3 default max_episode_steps is 200
            action = policy[observation]
            observation, reward, done, truncated, _ = env.step(action)
            episode_reward += reward
            episode_steps += 1
            if done or truncated:
                break
        
        total_rewards.append(episode_reward)
        total_steps.append(episode_steps)
        if done and reward == 20: # Successful dropoff gives +20
            successful_dropoffs += 1
            
    avg_reward = np.mean(total_rewards)
    std_reward = np.std(total_rewards)
    avg_steps = np.mean(total_steps)
    std_steps = np.std(total_steps)
    success_rate = (successful_dropoffs / num_episodes) * 100

    print(f"--- Evaluation Results ---")
    print(f"Average Reward: {avg_reward:.2f} +/- {std_reward:.2f}")
    print(f"Average Steps: {avg_steps:.2f} +/- {std_steps:.2f}")
    print(f"Success Rate: {success_rate:.2f}%")
    
    return avg_reward, avg_steps, success_rate, std_reward, std_steps

# === Mapping actions to text ===
action_names = {
    0: "South ↓", 1: "North ↑", 2: "East →",
    3: "West ←", 4: "Pickup :)", 5: "Dropoff ⬇"
}
locations = ["R (Red)", "G (Green)", "Y (Yellow)", "B (Blue)", "In Taxi"]


# === Main Execution ===
if __name__ == "__main__":
    env_pi = gym.make("Taxi-v3", render_mode=None)
    raw_env = env_pi.unwrapped     # now raw_env.P exists


    # ---  Run Policy Iteration and Collect Training Metrics ---
    print("--- Starting Policy Iteration ---")
    pi_start_time = time.time()
   
    optimal_policy, optimal_values, pi_iterations = policy_iteration(raw_env, discount_factor=0.99, theta=1e-6)

    pi_training_time = time.time() - pi_start_time
    print(f"Policy Iteration Training Time: {pi_training_time:.4f} seconds")
    print(f"Policy Iteration Converged in: {pi_iterations} iterations")

    # --- 2Evaluate the Final Policy from Policy Iteration ---
    pi_avg_reward, pi_avg_steps, pi_success_rate, pi_std_reward, pi_std_steps = evaluate_policy(env_pi, optimal_policy, num_episodes=1000)

    # ---  Store Metrics for Report ---
    policy_iteration_metrics = {
        "Algorithm": "Policy Iteration",
        "Iterations to Converge": pi_iterations,
        "Training Time (s)": pi_training_time,
        "Final Policy - Avg Reward": f"{pi_avg_reward:.2f} ± {pi_std_reward:.2f}",
        "Final Policy - Avg Steps": f"{pi_avg_steps:.2f} ± {pi_std_steps:.2f}",
        "Final Policy - Success Rate (%)": f"{pi_success_rate:.2f}",
       
        "Avg Reward per Ep (Learning Curve)": "N/A (Model-based)",
        "Steps per Ep (Learning Curve)": "N/A (Model-based)",
        "Success Rate (Learning Curve)": "N/A (Model-based)"
    }
    
    print("\n--- Policy Iteration Metrics Summary ---")
    for key, value in policy_iteration_metrics.items():
        print(f"{key}: {value}")

    # ---  Simulate one episode with the learned policy for visualization ---
    print("\n--- Simulating one episode with the optimal policy ---")
    observation, info = env_pi.reset(seed=42) 
    done = False
    truncated = False
    total_reward_sim = 0
    step_sim = 0

    while not done and not truncated and step_sim < 50: # Limit steps for display
        action = optimal_policy[observation]
        value_sim = optimal_values[observation]
        taxi_row, taxi_col, pass_idx, dest_idx = decode_taxi_state(observation)
        carrying = pass_idx == 4
        map_output = draw_custom_map(taxi_row, taxi_col, pass_idx, dest_idx, carrying)
        
        print(map_output)
        print(f"Step: {step_sim}, State ID: {observation} → (row={taxi_row}, col={taxi_col}, pass_loc={locations[pass_idx]}, dest={locations[dest_idx]})")
        print(f"Action: {action_names[action]} ({action}), Value Estimate: {value_sim:.2f}")
        
        observation, reward, done, truncated, info = env_pi.step(action)
        total_reward_sim += reward
        print(f"Reward: {reward}, Total Reward: {total_reward_sim}")
        print(f"{'-'*30}")
        time.sleep(0.5)
        step_sim += 1
    
    print(f"\nSimulation finished in {step_sim} steps with total reward: {total_reward_sim}")
    if done and reward == 20: print("Successful Dropoff in simulation!")
    elif truncated: print("Simulation truncated (max steps reached).")

    env_pi.close()

--- Starting Policy Iteration ---
Policy Iteration: Iteration 1 complete. Policy stable: False
Policy Iteration: Iteration 2 complete. Policy stable: False
Policy Iteration: Iteration 3 complete. Policy stable: False
Policy Iteration: Iteration 4 complete. Policy stable: False
Policy Iteration: Iteration 5 complete. Policy stable: False
Policy Iteration: Iteration 6 complete. Policy stable: False
Policy Iteration: Iteration 7 complete. Policy stable: False
Policy Iteration: Iteration 8 complete. Policy stable: False
Policy Iteration: Iteration 9 complete. Policy stable: False
Policy Iteration: Iteration 10 complete. Policy stable: False
Policy Iteration: Iteration 11 complete. Policy stable: False
Policy Iteration: Iteration 12 complete. Policy stable: False
Policy Iteration: Iteration 13 complete. Policy stable: False
Policy Iteration: Iteration 14 complete. Policy stable: False
Policy Iteration: Iteration 15 complete. Policy stable: False
Policy Iteration: Iteration 16 complete. Poli

Evaluating Policy: 100%|██████████| 1000/1000 [00:00<00:00, 1612.84it/s]


--- Evaluation Results ---
Average Reward: 7.72 +/- 2.56
Average Steps: 13.28 +/- 2.56
Success Rate: 100.00%

--- Policy Iteration Metrics Summary ---
Algorithm: Policy Iteration
Iterations to Converge: 17
Training Time (s): 3.0773661136627197
Final Policy - Avg Reward: 7.72 ± 2.56
Final Policy - Avg Steps: 13.28 ± 2.56
Final Policy - Success Rate (%): 100.00
Avg Reward per Ep (Learning Curve): N/A (Model-based)
Steps per Ep (Learning Curve): N/A (Model-based)
Success Rate (Learning Curve): N/A (Model-based)

--- Simulating one episode with the optimal policy ---
+---------+ 
|R: | : :G| 
| : | : : | 
| : : : : | 
| | : | : |T
|Y| : |B: | 
+---------+ 
Step: 0, State ID: 386 → (row=3, col=4, pass_loc=G (Green), dest=Y (Yellow))
Action: North ↑ (1), Value Estimate: 835.38
Reward: -1, Total Reward: -1
------------------------------
+---------+ 
|R: | : :G| 
| : | : : | 
| : : : : |T
| | : | : | 
|Y| : |B: | 
+---------+ 
Step: 1, State ID: 286 → (row=2, col=4, pass_loc=G (Green), dest=Y 